In [ ]:
import pandas as pd
import os, pathlib
import numpy as np
from import_clean import *
from calculate import *
from itertools import combinations

In [ ]:
all_data = cleanup_all(import_all_data("utla"), remove_geography_code=False)

In [ ]:
keys = list(all_data.keys())
for k1, k2 in combinations(keys, 2):
    df1, df2 = all_data[k1], all_data[k2]
    df1, df2 = df1[df1.columns.drop([])]
